# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [2]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'yeouido contracts_20210219095435.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [3]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [4]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [28]:


"""
contracts = {

}
pprint(contracts)
"""
pprint(contracts)

{'addressProvider': 'cx797bc63e7b27758816910cb35f0fa1f546b57c33',
 'daoFund': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
 'delegation': 'cx24e32367425660a67c91d6e3c1077914df4e0286',
 'feeProvider': 'cxd55f27870c9b219272ea4fc31d5d7ec8dc78a97e',
 'governance': 'cx672132f49f0ef02969cc6b223613206e46ff34b1',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'lendingPoolCore': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
 'lendingPoolDataProvider': 'cx88ae036091243916d87cbc3b11f39e317dcd46ad',
 'liquidationManager': 'cx21b6ee09cd144b2a778bdcf69a46ef2cf898ff1c',
 'oICX': 'cx9dfe6300788178da4ac43e7eb7114380f5addc69',
 'oIUSDC': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
 'oToken': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
 'oUSDb': 'cxa15c0e87237dfb3c7818078f34d371398b46fe29',
 'ommToken': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348',
 'priceOracle': 'cxe76fd817fae3b485ae6e4fea2cf88d050d379b63',
 'rewards': 'cx3e9b

# Importing wallet

### Importing from private key

In [6]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [7]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

16700592761150298909795

### Importing wallet from keystore file

In [8]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

708876575999000000000000000

### Creating new wallet

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

# Deploying all contracts


In [9]:
#contracts=AutoVivification()

deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token', 'sicx', 'liquidationManager']
# for directory in {"score"}:
directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "sicx":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[item] = res.get('scoreAddress', '')

#deploy oICX 
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

now = datetime.utcnow()

filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename=network+'_'+filename
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending tra

{'addressProvider': 'cxc595228d12cc267c748026a6629af41060654dab',
 'feeProvider': 'cxb76322c7df895a14e47bc14b4aa21566b3edc069',
 'lendingPool': 'cxf44d5c379d7b458fe4bba1ecb0f8fd4a54e9d65c',
 'lendingPoolCore': 'cx515613f46e2b333f6bdfa5d7fd1bf93fa3043e14',
 'lendingPoolDataProvider': 'cxfdff702158ae5bfce3f25a4c040b850d96e25af8',
 'oICX': 'cxf872138faf25dfc5a1216a7a85cad24b8172d988',
 'oToken': 'cx9732efc2d75905dfe4d8bc2d055c2cf226391d00',
 'priceOracle': 'cx1fbce991b255545bcd31634ab6beda5605201f46',
 'sample_token': 'cxc88281539407ee6d766e58d302f27c50d517783c',
 'sicx': 'cx400639d22b6bce4def6e7d7be2bc36e7a39192ee',
 'liquidationManager': 'cx847bb94d386b34af9ba86dfdcace651dbd5af822'}

In [11]:
now = datetime.utcnow()

filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename=network+'_'+filename
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)

saving contracts list as custom_yeouido_DevTestcontracts_20201210064637.pkl


## Save updated contract

In [46]:
#use this to update a pre existing pkl file
filename = 'updated20201125_' + contracts_file_custom
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

saving contracts list as updated20201125_custom_contracts_20201122161857.pkl


{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider',
  'SCORE': 'cx726e8bacc442256099dda50438c11d004215afc5'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool',
  'SCORE': 'cxbfd0ecb9892ca2e1ae3d0494d48f6618620b7e48'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider',
  'SCORE': 'cxe408b46a010ff5f82e9e45610a0aadb4ff2b073c'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore',
  'SCORE': 'cx3daa5c2517fdd44bdf37904e5e537eefafb4d69d'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken',
  'SCORE': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle',
  'SCORE': 'cx6703fdc7e0e65c5195d6dbfb59a838f76a9c19b1'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token'

# ICX transfer

In [17]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae")\
    .nid(3) \
    .nonce(100) \
    .value(1000000*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)


# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0x3c546494f559d4ea9d34534f13387c74aa80658c67ac2630ce1adaf514e1bc86',
 'blockHeight': 1425621,
 'blockHash': '0xc6a2b22340f3c2756027f1926f731d1eaa78ed6c65653e92e85f58e7ee2d2b66',
 'txIndex': 1,
 'to': 'hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae',
 'stepUsed': 100000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 100000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [25]:
params = {"_owner":"hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("balanceOf")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x2e044c431b4da18bd8d57bb81662e176a9e0c35d0258d0f8fdd9e064801d6738',
 'blockHeight': 10244800,
 'blockHash': '0x136d86ac10f8ae06e8b9c76f13b9074e34011902bbfe20314ac21d4215a77af9',
 'txIndex': 1,
 'to': 'cx93119bb51b7885b502293882004bbc5dc74e8215',
 'stepUsed': 142400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 142400,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [12]:
pprint(contracts)

{'addressProvider': 'cx797bc63e7b27758816910cb35f0fa1f546b57c33',
 'daoFund': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
 'delegation': 'cx24e32367425660a67c91d6e3c1077914df4e0286',
 'feeProvider': 'cxd55f27870c9b219272ea4fc31d5d7ec8dc78a97e',
 'governance': 'cx672132f49f0ef02969cc6b223613206e46ff34b1',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'lendingPoolCore': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
 'lendingPoolDataProvider': 'cx88ae036091243916d87cbc3b11f39e317dcd46ad',
 'liquidationManager': 'cx21b6ee09cd144b2a778bdcf69a46ef2cf898ff1c',
 'oICX': 'cx9dfe6300788178da4ac43e7eb7114380f5addc69',
 'oIUSDC': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
 'oToken': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
 'oUSDb': 'cxa15c0e87237dfb3c7818078f34d371398b46fe29',
 'ommToken': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348',
 'priceOracle': 'cxe76fd817fae3b485ae6e4fea2cf88d050d379b63',
 'rewards': 'cx3e9b

# Updating a contract

In [58]:
#deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token', 'sicx', 'oICX', 'liquidationManager']
#for contract_ in deploy:


contract_ = 'ommToken'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
elif contract_ == 'oUSDb':
    update = contracts['oUSDb']
    contract_ = 'oToken'
elif contract_ == 'oIUSDC':
    update = contracts['oIUSDC']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


update ommToken cxa96491850d5dd69efa5d64afa9138fd4a66cd348
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x54caadbb819efd425e144aa68380360c2bde0b300a279470a9af7f44034f143f',
 'blockHeight': 14379618,
 'blockHash': '0x506abb3e84707fe4cae1cc02ef7cc1a3e1b502099f5522d23186c2da6a335bf3',
 'txIndex': 1,
 'to': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348',
 'scoreAddress': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348',
 'stepUsed': 1794274400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1794274400,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [59]:
pprint(contracts)

{'addressProvider': 'cx797bc63e7b27758816910cb35f0fa1f546b57c33',
 'daoFund': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
 'delegation': 'cx24e32367425660a67c91d6e3c1077914df4e0286',
 'feeProvider': 'cxd55f27870c9b219272ea4fc31d5d7ec8dc78a97e',
 'governance': 'cx672132f49f0ef02969cc6b223613206e46ff34b1',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'lendingPoolCore': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
 'lendingPoolDataProvider': 'cx88ae036091243916d87cbc3b11f39e317dcd46ad',
 'liquidationManager': 'cx21b6ee09cd144b2a778bdcf69a46ef2cf898ff1c',
 'oICX': 'cx9dfe6300788178da4ac43e7eb7114380f5addc69',
 'oIUSDC': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
 'oToken': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
 'oUSDb': 'cxa15c0e87237dfb3c7818078f34d371398b46fe29',
 'ommToken': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348',
 'priceOracle': 'cxe76fd817fae3b485ae6e4fea2cf88d050d379b63',
 'rewards': 'cx3e9b

In [53]:
print(int("0x56bc75e2d63100000",0)/10**18)

100.0


In [54]:
params = {"_value": 150 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['ommToken']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .method("stake") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x451879e53693836eb082a4f5950abe17da959c51c4c629f2e91a658285b225ef',
 'blockHeight': 14314758,
 'blockHash': '0x2e4482cd6130d4199a3d71c40b8e65fd8ef86bace447d18ea72c4e69b34f6e50',
 'txIndex': 1,
 'to': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348',
 'stepUsed': 148040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 148040,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Depositing USDb from a wallet

In [109]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'txHash': '0xd61dd328e5dfa251632609893233dd383a15c76e680cf0b5f4d8751867e6340a',
 'blockHeight': 1353677,
 'blockHash': '0xa0b52c7c33c17a11f4d114baab6c9a18c87e500fd8e375cdaa00395c2b8aba9c',
 'txIndex': 1,
 'to': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
 'stepUsed': 751720,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 751720,
 'eventLogs': [{'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x0',
    '0x2386f26fc10000'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
    '0x0',
    '0x2386f26fc10000'],
   'data': ['0xde0b6b3a7640000', '0xd

# Borrowing USDb

In [36]:
params ={"_reserve": contracts['usdb'], "_amount":10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32006, 'message': "Type mismatch: name=_userData type_hint=<class '01837b298e3e5aae24b7ebcc093f168414c3babcb3.0x62819c6f01b36b7caa2670e22560a4398e0ef7b076961030fb668197d8696362.snapshot.UserSnapshotData'> value_type=<class 'dict'>"}


JSONRPCException: {'code': -32006, 'message': "Type mismatch: name=_userData type_hint=<class '01837b298e3e5aae24b7ebcc093f168414c3babcb3.0x62819c6f01b36b7caa2670e22560a4398e0ef7b076961030fb668197d8696362.snapshot.UserSnapshotData'> value_type=<class 'dict'>"} (Json rpc error)

# Reedeming/Withdrawing USDb

In [111]:
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65', 'stepLimit': '0x143ebc', 'timestamp': '0x5b553bf01f180', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'redeem', 'params': {'_amount': '0x8ac7230489e80000'}}, 'signature': 'A6J8pKu1O6z4b6+yRLpfqwyJ4duW7yxDUyS/z0tw5TRaYflRZ4z98yS881ACeFNCL+V/TjvTpJGOWm35iNIKoQE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x5b0cc66a6ff064bf2ecc8d9d50b0512570308396c11a958925a403a1ec357a5f',
 'blockHeight': 1353696,
 'blockHash': '0x2a41466d89cd9b869a0500d849a6422bf8e41065b272106100973ce157886430',
 'txIndex': 1,
 'to': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65',
 'stepUsed': 1226780,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1226780,
 'eventLogs': [{'scoreAddress': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0xf97c99928']},
  {'scoreAddress': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65',
   'indexed': ['Burn(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x8ac7230489e80000']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x0',
    '0x8ac7230489e80000'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed

# Repaying USDb

In [113]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 10000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 10000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x277fae145ad2b5aba3b4658c75f9c795b9cd26d2a014e803f440349f706b0ed5',
 'blockHeight': 1353714,
 'blockHash': '0x807be102e3e53321d0192a4fea9efc26acedd859277c2fcfe785651ebea33f5b',
 'txIndex': 1,
 'to': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
 'stepUsed': 929860,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 929860,
 'eventLogs': [{'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x8a6e51a672858000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x34ece5a8f5',
    '0x23887a690fba23'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
    '0x34ece5a8f5',
    '0x23887a690fba23'],
   'data': ['0xde0b6b

# Depositing ICX from Wallet

In [114]:
params = {"_amount": 500 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(500 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x5142a4c6b2ed37986f2e06689ec53d7e884a8cd7ccdced40f64e4f482261d568',
 'blockHeight': 1353718,
 'blockHash': '0x1ce2ce8efae957061913479ec271b5a504a5049d594020d57a18704130fe3de3',
 'txIndex': 1,
 'to': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
 'stepUsed': 571080,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 571080,
 'eventLogs': [{'scoreAddress': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
    'cx2d53c5e8347915733500bda8bdf2aabefdd833d9',
    '0x1b1ae4d6e2ef500000'],
   'data': []},
  {'scoreAddress': 'cx2d53c5e8347915733500bda8bdf2aabefdd833d9',
   'indexed': ['Mint(Address,int)',
    'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
    '0x1b1ae4d6e2ef500000'],
   'data': []},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x1b1ae4d6e2ef500000',
    '0x0'],
   'data'

# Withdrawing ICX 

In [115]:
params = {"_amount": 250 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX'])\
    .nid(3) \
    .step_limit(10000000) \
    .nonce(100) \
    .method("redeem") \
    .params(params) \
    .build()
signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)
        

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x4e18d42ba4451db152afef7509f2ab5c4cbb0efb3e9123ef47930d2a2641721d',
 'blockHeight': 1353723,
 'blockHash': '0x7e9f49501066c8329b270e447d413c104bab52118103d07edbfff08f78f7e25a',
 'txIndex': 1,
 'to': 'cx91a8f04a2a5adcf110168e68479e34a3b33424c9',
 'stepUsed': 1396820,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1396820,
 'eventLogs': [{'scoreAddress': 'cx91a8f04a2a5adcf110168e68479e34a3b33424c9',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cx91a8f04a2a5adcf110168e68479e34a3b33424c9',
   'indexed': ['Burn(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0xd8d726b7177a80000']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x0',
    '0xd8d726b7177a80000'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['P

# Borrowing sicx

In [30]:
#borrow sicx
params ={"_reserve": contracts['sicx'], "_amount":1*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32006, 'message': 'Too many arguments'}


JSONRPCException: {'code': -32006, 'message': 'Too many arguments'} (Json rpc error)

# Repay sicx

In [29]:

data = "{\"method\": \"repay\", \"params\": {\"amount\": 100000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 100000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x4b95d8683af8ba72d8ab4ba223c31b1c9c2f5e5b9ad262a4fa6f1a2ad45138f7',
 'blockHeight': 9908080,
 'blockHash': '0x51f885287ac717250a1775e9dc72600273acde6d78f3b4689fff9b3a40594a29',
 'txIndex': 1,
 'to': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
 'stepUsed': 867000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 867000,
 'eventLogs': [{'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x5684f308079370000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x9cdabb163294f',
    '0x433145f3bf1722'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
    '0x9cdabb163294f',
    '0x433145f3bf1722'],
   'data': ['0

## Minting  omm tokens

In [ ]:
params = {"_amount": 50 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['ommToken']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .method("testMint") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


## Staking omm tokens

In [ ]:
params = {"_value": 500 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['ommToken']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .method("stake") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


## Updating delegation preference of user

In [ ]:

params = {"_delegations":[{"_address":"hxd3be921dfe193cd49ed7494a53743044e3376cd3","_votes_in_per":str(6*10**17)},
                          {"_address":"hxec79e9c1c882632688f8c8f9a07832bcabe8be8f","_votes_in_per":str(4*10**17)}]}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['delegation']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .method("updateDelegations") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


# Readonly methods to get data

### Getting all the addresses

In [10]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['addressProvider'])\
    .method("getAllAddresses")\
    .build()
response = icon_service.call(_call)
pprint(response)

{'collateral': {'IUSDC': 'cx65f639254090820361da483df233f6d0e69af9b7',
                'USDb': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634',
                'sICX': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281'},
 'oTokens': {'oICX': 'cx04e21bb3d2e09b0e753af2dff0eeddb011a4be75',
             'oIUSDC': 'cx42a4734508068d0f8edd7ef0778b515788b3f3b8',
             'oUSDb': 'cx47549e03024584dca6eeb26b7de8f189bf2b6ff9'},
 'systemContract': {'Delegation': 'cx0ac8854f9fbe0126e830386046ed26bbb6769bf1',
                    'LendingPool': 'cx0287ab61ae0bec1af11ccb108257b7d292422d39',
                    'LendingPoolDataProvider': 'cx5b611522f541f77dfeab6a8f874e0e87db320a78',
                    'OmmToken': 'cx5dc583ebe474fd99db3962e32deff60c76328b61',
                    'Rewards': 'cx1c1d88f6039ebb04572d194d52e9b190f37514d6',
                    'Staking': 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c'}}


### Getting specific reserve data

In [63]:
#this returns the reserve data of USDb
params = {'_reserve': contracts['sicx']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': 884483729472659675269,
 'availableLiquidityUSD': 1622548221761714694568,
 'baseLTVasCollateral': 500000000000000000,
 'borrowCumulativeIndex': 1000000405752481418,
 'borrowRate': 6782998786072307,
 'borrowingEnabled': '0x1',
 'decimals': 18,
 'exchangePrice': 1797000000000000000,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': 1614765046165446,
 'liquidationBonus': 100000000000000000,
 'liquidationThreshold': 650000000000000000,
 'liquidityCumulativeIndex': 1000000073818905134,
 'liquidityRate': 414081652786726,
 'oTokenAddress': 'cx04e21bb3d2e09b0e753af2dff0eeddb011a4be75',
 'reserveAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'sICXRate': 1020844721329202557,
 'totalBorrows': 100000000000000000000,
 'totalBorrowsUSD': 183445796422857699500,
 'totalLiquidity': 984483729472659675269,
 'totalLiquidityUSD': 1805994018184572394068,
 'usageAsCollateralEnabled': '0x1'}


# Reserve Account Data

In [39]:
#this returns the reserve data of USDb

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveAccountData")\
    .build()
response = icon_service.call(_call)
# print(response)
for key,values in response.items():
    response[key]=int(values,0)/10**18
        

pprint(response)

{'availableLiquidityBalanceUSD': 0.0,
 'totalBorrowsBalanceUSD': 0.0,
 'totalCollateralBalanceUSD': 0.0,
 'totalLiquidityBalanceUSD': 0.0}


### Getting data of all reserves

In [159]:
#this returns the reserve data of all the reserves
params = {'_reserve': contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'availableLiquidity': 298500.0,
          'baseLTVasCollateral': 0.6,
          'borrowCumulativeIndex': 1.0000011584585715,
          'borrowRate': 0.08359381655340004,
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': 1606110048844793,
          'liquidationBonus': 1e-17,
          'liquidationThreshold': 0.65,
          'liquidityCumulativeIndex': 1.0000005791745306,
          'liquidityRate': 0.04716259038240734,
          'oTokenAddress': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe',
          'reserveAddress': 'cx134a1a0309aaf88a00ad1fee4983b7ed43454ab6',
          'totalBorrows': 501500.1141552619,
          'totalLiquidity': 800000.1141552619,
          'usageAsCollateralEnabled': '0x1'},
 'sICX': {'availableLiquidity': '0x0',
          'baseLTVasCollateral': 0.33,
          'borrowCumulativeIndex': 1.0,
          'borrowRate': '0x0',
          'borrowingEnabl

### Getting user account data

In [41]:
#returns the user data of all the reserves combined
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowingPower': 0.0,
 'currentLiquidationThreshold': 0.0,
 'currentLtv': 0.0,
 'healthFactor': -1e-18,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 0.0,
 'totalCollateralBalanceUSD': 0.0,
 'totalFeesUSD': 0.0,
 'totalLiquidityBalanceUSD': 0.0}


### Getting user reserve data

In [31]:
#returns the data of a specific user for a specific reserve
#this returns the data of user for USDb reserve
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":contracts['sicx']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.018912999951505187,
 'currentBorrowBalance': 40.500628612069235,
 'currentBorrowBalanceUSD': 20.250314306034618,
 'currentOTokenBalance': 250.00056435725696,
 'currentOTokenBalanceUSD': 125.00028217862848,
 'lastUpdateTimestamp': 1606843688248792,
 'liquidityRate': 0.002759412089563471,
 'originationFee': 0.0,
 'principalBorrowBalance': 40.50062730044531,
 'principalBorrowBalanceUSD': 20.250313650222655,
 'useAsCollateral': 1e-18,
 'userBorrowCumulativeIndex': 1.000011493399191}


# Getting All Reserve Data

In [44]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
pprint(response)

{'USDb': {'availableLiquidity': '0x3f35b7a91f3815900000',
          'baseLTVasCollateral': '0x853a0d2313c0000',
          'borrowCumulativeIndex': '0xde0b7c160e6fb0d',
          'borrowRate': '0x128fc221d15a6e5',
          'borrowingEnabled': '0x1',
          'decimals': '0x12',
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': '0x5b4bfa2ee7bf9',
          'liquidationBonus': '0xa',
          'liquidationThreshold': '0x905438e60010000',
          'liquidityCumulativeIndex': '0xde0b73a80e1fe23',
          'liquidityRate': '0xa78e20466e3ea9',
          'oTokenAddress': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe',
          'reserveAddress': 'cx134a1a0309aaf88a00ad1fee4983b7ed43454ab6',
          'totalBorrows': '0x6a32602b7ababa00e840',
          'totalLiquidity': '0xa96817d499f2cf90e840',
          'usageAsCollateralEnabled': '0x1'},
 'sICX': {'availableLiquidity': '0x0',
          'baseLTVasCollateral': '0x494654067e10000',
          'borrowCu

### Getting the data of user for all reserves

In [42]:
#returns the data of a specific user for all reserves

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'borrowRate': 0.0,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 0.0},
 'sICX': {'borrowRate': 0.0,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 0.0}}


## Getting user delegation preference

In [ ]:
params = {'_user':deployer_wallet.get_address()}
# params = {'_user':"hx81bf040426f226b3bfcd2f0b5967bbb0320525cf"}
_call = CallBuilder()\
    .from_("hx81bf040426f226b3bfcd2f0b5967bbb0320525cf")\
    .to(contracts['delegation'])\
    .method("getUserDelegationDetails")\
    .params(params)\
    .build()
response = icon_service.call(_call)
pprint(response)

## Getting delegation percentage for preps

In [ ]:

_call = CallBuilder()\
    .from_("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")\
    .to(contracts['delegation'])\
    .method("computeDelegationPercentages")\
    .build()
response = icon_service.call(_call)
data = []
for items in response:
    items['_votes_in_per']= int(items['_votes_in_per'],0) / 10**18
    data.append(items)

pprint(data)
# hxec79e9c1c882632688f8c8f9a07832bcabe8be8f
# hxec79e9c1c882632688f8c8f9a07832bcabe8be8f

## Getting top preps from testnet

In [ ]:
params = {"startRanking": 1,"endRanking":2}
_call = CallBuilder()\
    .from_("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")\
    .to(GOVERNANCE_ADDRESS)\
    .method("getPReps")\
    .params(params)\
    .build()
response = icon_service.call(_call)
pprint(response)
hxec79e9c1c882632688f8c8f9a07832bcabe8be8f
hxec79e9c1c882632688f8c8f9a07832bcabe8be8f